In [219]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

URL = "https://blueskymodel.org/air-mile"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")


# table3 --------------------------------------------
results = soup.find(id="table-3")
rows = results.find_all('tr')
some = []
for i in rows:
    datas = i.find_all('td')
    some.append(datas)
datas = rows[1].find_all('td')
aircraft = some[1:-1]
information = [
    [i.text for i in j] for j in aircraft
]
header = results.find_all('thead')
temp = []
for i in header:
    cols = i.find_all('th')
    temp.append(cols)
col_names = [temp[0][i].text for i in range(len(temp[0]))]
table3 = pd.DataFrame(data = information, columns = col_names)
table3 = table3.set_index('Model')

# print(table3)

In [220]:
# table 4
results = soup.find(id="table-4")
rows = results.find_all('tr')
some = []
for i in rows:
    datas = i.find_all('td')
    some.append(datas)
datas = rows[1].find_all('td')
aircraft = some[1:-1]
information = [
    [i.text for i in j] for j in aircraft
]
header = results.find_all('thead')
temp = []
for i in header:
    cols = i.find_all('th')
    temp.append(cols)
col_names = [temp[0][i].text for i in range(len(temp[0]))]
table4 = pd.DataFrame(data = information, columns = col_names)
table4 = table4.set_index('Model')
# print(table4)

In [221]:
# table 5
table5 = pd.DataFrame(columns = ['Model', 'as a % Boeing planes', 'Avg. fuel efficiency (gal/mi)', 'Avg. passengers'])
a = pd.Series(table3.index)
b = pd.Series(table3['as % Boeing planes'])
b = b[:-1]
b[0] = 37.7
table5['Model'] = a[0:5]
norm = [float(i) for i in b.values]

table5 = table5.set_index('Model')
table5['as a % Boeing planes'] = norm

fueleff = [1.79, 6.61, 2.75, 2.75, 5.29]
passengers = [129, 486, 222, 266, 383]
table5["Avg. fuel efficiency (gal/mi)"] = fueleff
table5['Avg. passengers'] = passengers

table5['weights'] = table5['as a % Boeing planes'] * table5['Avg. fuel efficiency (gal/mi)']

fuel_eff_avg = table5['weights'].sum()/100

# efficiency of an average aircraft (gal per mi)
dodecane_dens = 2839.06
co2produced = 3.1
conversion = 1/453.59
kmmi = 1.60934
co2emitted = dodecane_dens * fuel_eff_avg * co2produced * conversion / kmmi
# lbs of co2 emitted per mile travelled in an average aircraft
print('An average aircraft emits an average of {} lbs of co2 per 1 km travelled'.format(co2emitted))

An average aircraft emits an average of 33.2272765287365 lbs of co2 per 1 km travelled


In [222]:
from geopy.geocoders import Nominatim
from geopy import distance

geocoder = Nominatim(user_agent = 'I know python')
location1 = str(input('Location1:'))
location2 = str(input('Location2:'))

coordinates1 = geocoder.geocode(location1)
coordinates2 = geocoder.geocode(location2)

lat1, long1 = (coordinates1.latitude),(coordinates1.longitude)
lat2, long2 = (coordinates2.latitude),(coordinates2.longitude)

place1 = (lat1, long1)
place2 = (lat2, long2)
distance = distance.distance(place1, place2)
print('The distance between {0} and {1} is {2}'.format(location1, location2, distance))


distance = str(distance)[:-4]
distance = float(distance)

result_emission = co2emitted * distance
print('An average plane travelling between {} and {} would emit on average of {} pounds of co2'.format(location1, location2, result_emission))

Location1:Busan
Location2:Darwin
The distance between Busan and Darwin is 5275.868967770384 km
An average plane travelling between Busan and Darwin would emit on average of 175302.757121486 pounds of co2
